In [12]:
import os
! cd .. && python -m pip install --user .

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /home/jsteinbauer/ondewo/ondewo-s2t-client-python
     |████████████████████████████████| 4.0 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 672 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 10.4 MB/s ta 0:00:011
  Created wheel for ondewo-s2t-client: filename=ondewo_s2t_client-3.1.1-py2.py3-none-any.whl size=20316 sha256=96775f78f2f5fbcdb0101707b68dd303bd04baa1c45c6e117f8e8b6bd5b9eb7a
  Stored in directory: /tmp/pip-ephem-wheel-cache-k3p96bfe/wheels/68/33/5c/6741336a8de21fa9ea1d4ff5e96fc434fb52c5c39721dc69c7
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3576 sha256=4f6e4de22878147494d267a1ea17f5f5c50431f961fe4d846b5c885f32fb2b91

In [15]:
! cd ..

In [18]:
import ondewo.s2t.client

ModuleNotFoundError: No module named 'ondewo'

In [1]:
import argparse
import wave

from ondewo.s2t import speech_to_text_pb2
from ondewo.s2t.client.client import Client
from ondewo.s2t.client.client_config import ClientConfig
from ondewo.s2t.client.services.speech_to_text import Speech2Text
from ondewo.s2t.speech_to_text_pb2 import ListS2tPipelinesRequest, Speech2TextConfig

In [2]:
import ondewo_s2t_client

ModuleNotFoundError: No module named 'ondewo_s2t_client'

In [3]:
lr = speech_to_text_pb2.ListS2tPipelinesRequest()

In [4]:
lr.registered_only

AttributeError: registered_only

## Creating a client object
The example below shows how to create a speech to text client object from a client config. 
When setting *use_secure_channel=True*, a grpc certificate *grpc_cert* is required.

In [4]:
AUDIO_FILE: str = "examples/audiofiles/sample_1.wav"
# credentials = grpc.ssl_channel_credentials(root_certificates=cert)

MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "localhost"
GRPC_PORT: str = "50655"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"
grpc_cert: str = None


options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]

# channel = grpc.secure_channel(CHANNEL, credentials, options=options)

config: ClientConfig = ClientConfig(
  host=GRPC_HOST,
  port=GRPC_PORT, 
  grpc_cert=grpc_cert)
    
print(config)
    
client: Client = Client(config=config, use_secure_channel=False)

ClientConfig(host='localhost', port='50655', grpc_cert=None)


## List all existing speech to text pipelines
All relevant configurations of the speech to text server are defined in a speech to text pipeline. 
A running server can store several such configs at the same time, such that the client can chose which one to 
pick when he sends a request to transcribe an audio file or stream.

The example below shows how to list all available pipelines by calling the **s2t_service.list_s2t_pipelines()** function, which takes a **ListS2tPipelinesRequest()** as an argument.


In [6]:
s2t_service: Speech2Text = client.services.speech_to_text
pipelines = s2t_service.list_s2t_pipelines(request=ListS2tPipelinesRequest()).pipeline_configs
for pipeline in pipelines:
    print(pipeline)

id: "ato001"
description {
  language: "de"
  pipeline_owner: "ondewo"
  domain: "itsupport"
}
active: true
inference {
  ctc_acoustic_models {
    type: "wav2vec"
    quartznet {
      config_path: "acoustic_models/model_configs/de/quartznet15x5_de.yaml"
      load_type: "pt_files"
      pt_files {
        path: "acoustic_models/de/quartznet15x5_de_2/"
        step: "2500"
      }
      ckpt_file {
        path: "acoustic_models/de/ckpt_test/test.ckpt"
      }
      use_gpu: true
    }
    quartznet_triton {
      config_path: "acoustic_models/model_configs/de/quartznet15x5_de.yaml"
      triton_url: "localhost:8001"
      triton_model: "quartznet"
    }
    wav2vec {
      model_path: "acoustic_models/de/wav2vec2_large_1360h_telephonized/"
      use_gpu: true
    }
  }
  language_models {
    path: "language_models/"
    beam_size: 64
    default_lm: "german_lm_big.binary"
    beam_search_scorer_alpha: 2.0
    beam_search_scorer_beta: 1.0
  }
}
streaming_server {
  output_style: "sim

## GetS2TPipeline, DeleteS2TPipeline, CreateS2TPipeline, UpdateS2TPipeline
In the following, we demonstrate how to do CRUD (Create, Retrieve, Update, Delete) pipelines.

The example below shows how to list all available pipelines by calling the **s2t_service.get_s2t_pipeline()** function, which takes a **S2tPipelineId** as an argument.


### GetS2TPipeline
To get a specific s2t pipeline, we can use the GetS2TPipeline endpoint.

In [7]:
from ondewo.s2t.speech_to_text_pb2 import S2tPipelineId

# ### GetS2TPipeline
# To get a specific s2t pipeline, we can use the GetS2TPipeline endpoint.

# pipeline_id = 'wav2vec_en'
# pipeline = s2t_service.get_s2t_pipeline(request=S2tPipelineId(id=pipeline_id))

pipelines = s2t_service.list_s2t_pipelines(request=ListS2tPipelinesRequest(registered_only=True)).pipeline_configs
print(f"Number of pipelines: {len(pipelines)}")

# print(pipeline)


# ### DeleteS2TPipeline
# To delete specific s2t pipeline, we can use the GetS2TPipeline endpoint.

# pipeline_id = 'wav2vec_en'
# pipeline = s2t_service.delete_s2t_pipeline(request=S2tPipelineId(id=pipeline_id))

# pipelines = s2t_service.list_s2t_pipelines(request=ListS2tPipelinesRequest()).pipeline_configs
# print(f"Number of pipelines after deletion: {len(pipelines)}")


# ### CreateS2TPipeline
# # To delete specific s2t pipeline, we can use the GetS2TPipeline endpoint.

# pipeline = s2t_service.create_s2t_pipeline(request=pipeline)

# pipelines = s2t_service.list_s2t_pipelines(request=ListS2tPipelinesRequest()).pipeline_configs
# print(f"Number of pipelines after deletion: {len(pipelines)}")

ValueError: Protocol message ListS2tPipelinesRequest has no "registered_only" field.

In [ ]:
lr = ListS2tPipelinesRequest()

In [ ]:
lr.registered_only

In [ ]:
from typing import List, Optional

def find_pipeline_for_language(pipelines: List[Speech2TextConfig], language: str) -> Optional[Speech2TextConfig]:
    """ 
    Returns the first speech to text pipeline for the requested language. 
    If no pipline is found, return None.
    """
    for pipeline in pipelines:
        if pipeline.description.language == language:
            return pipeline

english_pipeline = find_pipeline_for_language(pipelines=pipelines, language='en')
german_pipeline = find_pipeline_for_language(pipelines=pipelines, language='de')

## Load in example audio file
This audio file will be used in the following transcription examples.

In [ ]:
import wave 

audio_file_path = "audiofiles/sample_1.wav"

with wave.open(audio_file_path) as w:
    audio: bytes = w.readframes(w.getnframes())

## Send a transcribe file request to the server
In general, are two different endpoints for audio transcriptions:
1. **Transcribe an audio file** 
2. **Transcribe an audio stream**

### Transcribe an audio file
In this example, we create a **TranscribeFileRequest**, including the audio file (as bytes) and a **TranscribeRequestConfig** message, including the speech to text pipeline id, as well as optional additional parameters.
The request message is then used as an argument to the **s2t_service.transcribe_file()**, which calls the corresponding endpoint.

In [ ]:
import time
pipeline: Speech2TextConfig = pipelines[1]

request = speech_to_text_pb2.TranscribeFileRequest(
    audio_file=audio,
    config=speech_to_text_pb2.TranscribeRequestConfig(
        s2t_pipeline_id=pipeline.id,
        ctc_decoding=speech_to_text_pb2.CTCDecoding.BEAM_SEARCH_WITH_LM,
    )
)
# Send transcription request and get response
transcribe_response = s2t_service.transcribe_file(request=request)

for transcription_message in transcribe_response.transcriptions:
    print(f"File transcript: {transcription_message.transcription}")

### Transcribe an audio stream
In this example, we transcribe an audio stream by streaming a **TranscribeStreamRequest**, including audio chunks (as bytes) and a **TranscribeRequestConfig** message, including the speech to text pipeline id, as well as optional additional parameters.
The request message generator is then used as an argument to the **s2t_service.transcribe_stream()**, which calls the corresponding endpoint.

**Important**: After the TranscribeRequestConfig has been set once, it does not have to be sent with each new streameing request (this can help to save bandwidth). The old TranscribeRequestConfig remains until a new one is sent.

### Option 1.: Transcribe full utterances only (default)
In this mode, audio chunks are concatenated until a full utterance is accumulated (an utterance is considered "finished" if no voice is detected in the audio signal for `end_of_utterance_threshold` seconds.

In [ ]:
from typing import Iterator
from streaming_example import get_streaming_audio, create_streaming_request

# Get audio stream (iterator of audio chunks)
audio_stream: Iterator[bytes] = get_streaming_audio("audiofiles/sample_1.wav")

# Create streaming request
streaming_request: Iterator[speech_to_text_pb2.TranscribeStreamRequest] = create_streaming_request(
    audio_stream=audio_stream, 
    pipeline_id=pipeline.id,
    transcribe_not_final=False,
)

# Transcribe the stream and get back responses
response_gen: Iterator[speech_to_text_pb2.TranscribeStreamResponse] = s2t_service.transcribe_stream(
    request_iterator=streaming_request
)

# Print transcribed utterances
for i, response_chunk in enumerate(response_gen):
    for transcribe_message in response_chunk.transcriptions:
        print(f"{i}. response_chunk: {transcribe_message.transcription}")

### Option 1.: Transcribe not final
In this mode, audio chunks are transcribed as soon as a minimal length of voice signal is accumulated.

In [ ]:
from typing import Iterator
from streaming_example import get_streaming_audio, create_streaming_request

# Get audio stream (iterator of audio chunks)
audio_stream: Iterator[bytes] = get_streaming_audio("audiofiles/sample_1.wav")

# Create streaming request
streaming_request: Iterator[speech_to_text_pb2.TranscribeStreamRequest] = create_streaming_request(
    audio_stream=audio_stream, 
    pipeline_id=pipeline.id, 
    transcribe_not_final=True,
)

# Transcribe the stream and get back responses
response_gen: Iterator[speech_to_text_pb2.TranscribeStreamResponse] = s2t_service.transcribe_stream(
    request_iterator=streaming_request
)

# Print transcribed utterances
for i, response_chunk in enumerate(response_gen):
    for transcribe_message in response_chunk.transcriptions:
        print(f"{i}. response_chunk: {transcribe_message.transcription}")